<a href="https://colab.research.google.com/github/clevyy/Segmentation-des-communes-ile-france/blob/main/Segmentation_des_communes_d%E2%80%99ile_de_France.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use case ML

---



### Open in Colab to see the plots.

In [1]:
! pip install -q --upgrade plotly
! pip install -q jupyter-dash

In [2]:
! pip install -q kneed
! pip install -q gdown

# Importation des classes nécessaires pour notre étude.

In [3]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from kneed import KneeLocator # sert à détecter l'hyper-parametre K pour la méthode KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import silhouette_score

# Chargement des données

In [4]:
! gdown --id 1f3saJ6cefP_UUSZ4Qi9M488rNrU6nSS1

Downloading...
From: https://drive.google.com/uc?id=1f3saJ6cefP_UUSZ4Qi9M488rNrU6nSS1
To: /content/base-comparateur-de-territoires.csv
100% 2.43M/2.43M [00:00<00:00, 38.5MB/s]


In [5]:
df = pd.read_csv('base-comparateur-de-territoires.csv', sep = ";")

# Exploration

In [6]:
df.shape

(1300, 38)

In [7]:
df.head()

,CODGEO,LIBGEO,REG,DEP,P14_POP,P09_POP,SUPERF,NAIS0914,DECE0914,P14_MEN,NAISD16,DECESD16,P14_LOG,P14_RP,P14_RSECOCC,P14_LOGVAC,P14_RP_PROP,NBMENFISC14,PIMP14,MED14,TP6014,P14_EMPLT,P14_EMPLT_SAL,P09_EMPLT,P14_POP1564,P14_CHOM1564,P14_ACT1564,ETTOT15,ETAZ15,ETBE15,ETFZ15,ETGU15,ETGZ15,ETOQ15,ETTEF115,ETTEFP1015,Geo Shape,geo_point_2d
0,95555,Saint-Gratien,11,95,20996.0,20258.0,2.42,1564.0,749.0,8909.817815,257.0,156.0,9337.392497,8909.817815,56.919890,370.654792,4479.402681,8217.0,67.761158,21552.857143,14.972881,3831.135105,3224.519672,3516.342843,13417.955458,1387.832340,10146.844242,1475.0,3.0,38.0,142.0,1078.0,225.0,214.0,335.0,67.0,"{""type"": ""Polygon"", ""coordinates"": [[[2.274621...","48.9695044801, 2.28470111182"
1,95488,Pierrelaye,11,95,8155.0,7920.0,9.21,639.0,243.0,2891.000000,152.0,46.0,3086.000000,2891.000000,11.000000,184.000000,1591.000000,2903.0,66.586290,21415.277778,14.387392,3039.646367,2661.250427,2898.932365,5476.000000,509.000000,4044.000000,878.0,8.0,47.0,135.0,623.0,241.0,65.0,228.0,72.0,"{""type"": ""Polygon"", ""coordinates"": [[[2.193617...","49.0190794714, 2.1606870483"
2,95543,Saint-Cyr-en-Arthies,11,95,242.0,229.0,3.89,9.0,4.0,93.154812,1.0,1.0,106.271929,93.154812,7.063063,6.054054,86.066946,96.0,NaN,25937.142857,NaN,25.112024,18.060819,16.867783,141.757322,9.112971,109.355649,13.0,2.0,0.0,2.0,5.0,2.0,4.0,4.0,0.0,"{""type"": ""Polygon"", ""coordinates"": [[[1.732068...","49.058878265, 1.74176317637"
3,95523,La Roche-Guyon,11,95,446.0,447.0,4.61,18.0,21.0,186.507593,0.0,6.0,267.570530,186.507593,49.216783,31.846154,113.442763,185.0,NaN,21100.000000,NaN,372.590626,338.427249,505.753210,280.164177,24.995863,204.811694,74.0,0.0,6.0,6.0,48.0,17.0,14.0,13.0,4.0,"{""type"": ""Polygon"", ""coordinates"": [[[1.623323...","49.0883415268, 1.63442788467"
4,95678,Villiers-Adam,11,95,838.0,828.0,9.82,45.0,29.0,340.831981,5.0,9.0,364.250976,340.831981,9.860629,13.558365,299.077636,327.0,NaN,28242.000000,NaN,102.752654,83.329975,78.177577,516.588644,33.986095,400.064890,75.0,6.0,3.0,8.0,53.0,12.0,5.0,14.0,2.0,"{""type"": ""Polygon"", ""coordinates"": [[[2.236020...","49.0697597649, 2.23847351378"


In [8]:
df.columns

Index(['CODGEO', 'LIBGEO', 'REG', 'DEP', 'P14_POP', 'P09_POP', 'SUPERF',
       'NAIS0914', 'DECE0914', 'P14_MEN', 'NAISD16', 'DECESD16', 'P14_LOG',
       'P14_RP', 'P14_RSECOCC', 'P14_LOGVAC', 'P14_RP_PROP', 'NBMENFISC14',
       'PIMP14', 'MED14', 'TP6014', 'P14_EMPLT', 'P14_EMPLT_SAL', 'P09_EMPLT',
       'P14_POP1564', 'P14_CHOM1564', 'P14_ACT1564', 'ETTOT15', 'ETAZ15',
       'ETBE15', 'ETFZ15', 'ETGU15', 'ETGZ15', 'ETOQ15', 'ETTEF115',
       'ETTEFP1015', 'Geo Shape', 'geo_point_2d'],
      dtype='object')

On a 3 observations qui n'ont pas des coordonnées, on ne peut pas les donner des valeurs aléatoires.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CODGEO         1300 non-null   int64  
 1   LIBGEO         1300 non-null   object 
 2   REG            1300 non-null   int64  
 3   DEP            1300 non-null   int64  
 4   P14_POP        1297 non-null   float64
 5   P09_POP        1297 non-null   float64
 6   SUPERF         1297 non-null   float64
 7   NAIS0914       1297 non-null   float64
 8   DECE0914       1297 non-null   float64
 9   P14_MEN        1297 non-null   float64
 10  NAISD16        1296 non-null   float64
 11  DECESD16       1296 non-null   float64
 12  P14_LOG        1297 non-null   float64
 13  P14_RP         1297 non-null   float64
 14  P14_RSECOCC    1297 non-null   float64
 15  P14_LOGVAC     1297 non-null   float64
 16  P14_RP_PROP    1297 non-null   float64
 17  NBMENFISC14    1280 non-null   float64
 18  PIMP14  

On doit les supprimer.

In [10]:
df.dropna(subset = ['geo_point_2d'], inplace = True)

On détache le libelé et les coordonnées pour les utiliser après.

In [11]:
libele = df['LIBGEO']
lat = [float(x.split(",")[0]) for x in df['geo_point_2d'] if not pd.isnull(x)]
lon = [float(x.split(",")[1]) for x in df['geo_point_2d'] if not pd.isnull(x)]

Suppréssion des features inutiles (on est interessé par les données sociodémographiques)

In [12]:
df.drop(['CODGEO', 'LIBGEO', 'DEP', 'REG', 'Geo Shape', 'geo_point_2d'], axis = 1, inplace = True)

Il reste encore des valeurs manquants.

In [13]:
df.isna().sum()

P14_POP            0
P09_POP            0
SUPERF             0
NAIS0914           0
DECE0914           0
P14_MEN            0
NAISD16            1
DECESD16           1
P14_LOG            0
P14_RP             0
P14_RSECOCC        0
P14_LOGVAC         0
P14_RP_PROP        0
NBMENFISC14       19
PIMP14           736
MED14             19
TP6014           835
P14_EMPLT          0
P14_EMPLT_SAL      0
P09_EMPLT          0
P14_POP1564        0
P14_CHOM1564       0
P14_ACT1564        0
ETTOT15            0
ETAZ15             0
ETBE15             0
ETFZ15             0
ETGU15             0
ETGZ15             0
ETOQ15             0
ETTEF115           0
ETTEFP1015         0
dtype: int64

Suppréssion des features qui ont un pourcentage des valeurs manquantes supérieur à 30%.
* Pandas.count() -> compte nombre de valeurs non nulles d'une colonne.

In [14]:
nrows = df.shape[0]
for col in df.columns:
  if ( nrows - df[col].count() ) / nrows > 0.3:
    print(f'La colonne {col} a été supprimé.')
    df.drop(col, axis = 1, inplace = True)

La colonne PIMP14 a été supprimé.
La colonne TP6014 a été supprimé.


Remplissage des valeurs manquantes par la méthode KNNImputer.

In [15]:
for col in df.columns:
  imputer = KNNImputer(missing_values = np.nan, n_neighbors = 3)
  df[col] = imputer.fit_transform(df[col].values.reshape(-1, 1))

In [16]:
df.describe()

,P14_POP,P09_POP,SUPERF,NAIS0914,DECE0914,P14_MEN,NAISD16,DECESD16,P14_LOG,P14_RP,P14_RSECOCC,P14_LOGVAC,P14_RP_PROP,NBMENFISC14,MED14,P14_EMPLT,P14_EMPLT_SAL,P09_EMPLT,P14_POP1564,P14_CHOM1564,P14_ACT1564,ETTOT15,ETAZ15,ETBE15,ETFZ15,ETGU15,ETGZ15,ETOQ15,ETTEF115,ETTEFP1015
count,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.00000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000,1297.000000
mean,9273.373169,9042.590594,9.261581,701.562066,275.464919,3911.857733,137.33179,57.577932,4332.394204,3911.857733,145.102265,275.434206,1855.036807,3791.279343,24944.511311,4379.668431,3910.950009,4355.524536,6163.819807,592.009632,4693.056905,1076.016962,5.515806,35.397841,90.811874,824.212028,156.500386,120.079414,238.119507,59.921357
std,21504.336745,21245.410301,7.744994,1683.059982,656.938707,10119.507537,328.57665,133.775113,11531.301129,10119.507537,741.856491,856.974027,3753.704578,9342.236424,4199.028206,14384.431564,12800.822554,14391.617593,14668.142142,1557.838238,11262.165941,3946.507844,5.930795,109.232291,227.351478,3309.927593,459.764654,372.842179,885.875447,196.368772
min,24.000000,28.000000,0.090000,0.000000,0.000000,12.000000,0.00000,0.000000,15.000000,12.000000,0.000000,0.000000,0.960000,38.000000,13060.000000,3.900000,0.993289,4.583333,17.280000,0.000000,13.440000,6.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,549.000000,541.000000,4.720000,28.000000,14.000000,208.787565,5.00000,3.000000,242.823557,208.787565,9.613387,14.000000,175.307839,218.000000,22426.956522,68.881807,47.086480,65.182683,364.772059,22.364429,280.049541,46.000000,2.000000,2.000000,6.000000,27.000000,6.000000,4.000000,9.000000,1.000000
50%,1379.000000,1335.000000,7.650000,74.000000,39.000000,522.000000,14.00000,9.000000,583.809434,522.000000,22.000000,37.284829,417.517451,544.000000,24753.888889,276.117232,230.173358,269.806521,889.302141,56.135922,686.258374,116.000000,4.000000,6.000000,16.000000,74.000000,18.000000,13.000000,23.000000,5.000000
75%,7059.000000,6840.000000,11.670000,415.000000,215.000000,2749.000000,81.00000,50.000000,2928.087153,2749.000000,56.910569,165.619063,1831.000000,2981.000000,26954.000000,2093.789501,1798.026247,2013.200350,4645.399656,340.438625,3481.316900,590.000000,7.000000,26.000000,65.000000,412.000000,99.000000,75.000000,126.000000,35.000000
max,235366.000000,236491.000000,172.050000,16635.000000,7579.000000,125554.284324,3082.00000,1527.000000,149161.892457,125554.284324,11780.713252,11826.894881,39991.109284,113177.000000,42765.600000,168029.871732,148142.662065,164883.898410,163261.981373,16205.968043,127035.648749,65177.000000,87.000000,1377.000000,2515.000000,58871.000000,5517.000000,4629.000000,13888.000000,3537.000000


Box plot pour vérifier l'existence des valeurs outliers.
- On remarque qu'il existe des outliers dans toutes les colonnes.

In [17]:
px.box(df)

Utilisation de la méthode 'Interquartile range' (IQR) pour l'élimination des outliers.
* numpy.where(condition, x, y) 
  * si le condition est vrai retourne x sinon y.

In [18]:
for col in df.columns:
  q25, q75 = np.percentile(df[col].dropna(), 25), np.percentile(df[col].dropna(), 75)
  iqr = q75 - q25
  upper_limit = q75 + 1.5 * iqr
  lower_limit = q25 - 1.5 * iqr
  df[col] = np.where(
      df[col] < lower_limit,
      lower_limit,
      np.where(
          df[col] > upper_limit,
          upper_limit,
          df[col]
      )
  )

Les outliers ont été supprimés avec succès.
- Les features ont des unités différent, on doit les standariser.

In [19]:
px.box(df)

# Test de corrélation

Heat map des corrélations entre les features.
- On remarque que la plupart des features sont fortement corrélés entre eux.
 - On doit réduire la dimensionnalité.

In [20]:
go.Figure(
    data = go.Heatmap(
        z = df.corr(), x = df.columns, y = df.columns
    )
)

# Standarisation

Méthode de z-score.
- (Xi - moyen) / écart-type

In [21]:
df_scaled = StandardScaler().fit_transform(df)

# ACP

Réduction de dimensionnalité.

In [22]:
df_pca = PCA().fit(df_scaled)

Calcul de la somme cummulé des variances exprimés par les features afin de détecter le nombre de composants qui expriment le plus part d'information.

In [23]:
cumsum_variance = np.cumsum(df_pca.explained_variance_ratio_)

Plot de sommes cummulés des variances.
- kteb lih comenter a amhil rak tema (3zelna nb components = 2 hitach qui exprimer ~95% dyl l'information)

In [24]:
px.line(cumsum_variance)

ACP avec 2 composants principales.

In [25]:
df_pca = PCA(n_components=2).fit_transform(df)

In [26]:
df_pca

array([[22409.39138061, -1410.82673054],
       [ 6573.70477091, -2920.19784266],
       [-8750.55221929,   339.4359876 ],
       ...,
       [-8867.80578712,   686.8642852 ],
       [-8759.97224445,  1570.81147175],
       [-7704.94383356,  1064.54510049]])



> # KMeans



### Détection de la meilleur valeur d'hyper-parametre K

inertie = somme carré des distances entre les observations
- On sauvegarde les models correpondants aux différents valeurs de K pour ne pas les calculer à nouveau lors après le choix de K optimale.

In [27]:
nb_clusters = range(2, 31)
inertie = []
models = []
for K in nb_clusters:
  kmeans = KMeans(K)
  kmeans.fit(df_pca)
  models.append(kmeans)
  inertie.append(kmeans.inertia_)

Elbow plot.
- La meilleure valeur de K est comprise entre 7 et 10.

In [28]:
px.line(
    x = nb_clusters,
    y = inertie,
    labels = {
        "x": "nombre de clusters (K)",
        "y": "inertie"
    }
)

Détection numérique de 'Elbow point' avec la fonction KneeLocator du package 'kneed'.

In [29]:
kl = KneeLocator(nb_clusters, inertie, S=1.0, curve='convex', direction='decreasing')
K = kl.elbow
print(f'La valeur optimale de K est {K}.')

La valeur optimale de K est 8.


**range(2, ..)** donc le model correspondant est K - 2.

In [30]:
kmeans = models[K - 2]

Les centres des clusters.

In [31]:
kmeans.cluster_centers_

array([[10140.70192671, -1340.85520178],
       [-1437.76162929, -1172.80167862],
       [22599.84497054, -3148.28724692],
       [-7582.45156398, -3231.75088741],
       [-7283.42223092,  5988.59031868],
       [21739.58105753,  5865.43449545],
       [ 3456.5979783 ,  3940.08121003],
       [-7546.14421618,   548.30008596]])

Les classes.

In [32]:
classes_kmeans = kmeans.labels_
classes_kmeans

array([2, 0, 7, ..., 7, 7, 7], dtype=int32)


> # CAH



Dendrogram
- Le K est le nombre des lignes verticales qui traversent la coupe horizontale correspondant au plus grand saut entre deux clusters consécutifs.
- Ward's Linkage Method. Instead of measuring the distance directly, it analyzes the variance of clusters. Ward's is said to be the most suitable method for quantitative variables.

In [33]:
ff.create_dendrogram(
    df_pca,
    linkagefun = lambda x: linkage(x, "ward")
)

CAH avec 5 clusters.

In [34]:
cah = AgglomerativeClustering(n_clusters = 5).fit(df_pca)

Les classes.

In [35]:
classes_cah = cah.labels_
classes_cah

array([2, 0, 1, ..., 1, 1, 1])


> # Comparaison entre KMeans et CAH






- On ne peut pas évaluer nos models puisqu'on n'a pas des classes (labels) réels pour notre données.
 - Donc Il faut comparer le coefficient silhouette des deux clusters générés par les deux models.
   - The silhouette method computes silhouette coefficients of each point that measure how much a point is similar to its own cluster compared to other clusters.

Le coefficient silhouette de CAH.

In [36]:
silhouette_score(df_pca, classes_cah)

0.3928876334058058

Le coefficient silhouette de KMeans.

In [37]:
silhouette_score(df_pca, kmeans.labels_)

0.43894491288357024

Le coefficient de la méthode KMeans est plus grand que celui de CAH.
- On va utiliser le KMeans pour le regroupement des communes.

Assemblage de données.

In [38]:
communes = pd.DataFrame(
    {
      'component1': df_pca[:, 0],
      'component2': df_pca[:, 1],
      'classe': kmeans.labels_,
      'libele': libele,
      'lat': lat,
      'lon': lon
    }
)

In [39]:
communes.head()

,component1,component2,classe,libele,lat,lon
0,22409.391381,-1410.826731,2,Saint-Gratien,48.969504,2.284701
1,6573.704771,-2920.197843,0,Pierrelaye,49.019079,2.160687
2,-8750.552219,339.435988,7,Saint-Cyr-en-Arthies,49.058878,1.741763
3,-7776.607640,-4433.874830,3,La Roche-Guyon,49.088342,1.634428
4,-7846.987653,2730.979118,7,Villiers-Adam,49.069760,2.238474


10 communes qui ont des caractéristiques communs et qui appartient au 1er groupement.

In [40]:
communes[communes['classe'] == 0]['libele'].tolist()[:10]

['Pierrelaye',
 'Beauchamp',
 'Chanteloup-les-Vignes',
 'Magny-les-Hameaux',
 'Breuillet',
 'Épinay-sur-Orge',
 'Enghien-les-Bains',
 'Quincy-sous-Sénart',
 'Thorigny-sur-Marne',
 'Fontenay-le-Fleury']


> # Visualisation



Visualisation des communes avec le plot à nuage de points.

In [41]:
px.scatter(
    communes,
    x = 'component1',
    y = 'component2',
    color = 'classe',
    hover_data = ['libele']
)

Visualisation des communes dans un Map.
- color = appartenance à une classe.

In [42]:
px.scatter_mapbox(
    communes,
    lat = "lat",
    lon = "lon",
    color = "classe",
    hover_data = ['libele'],
    size_max = 20,
    zoom = 8,
    mapbox_style = "carto-positron"
)

Map plus sophistiqué.

In [43]:
fig = px.scatter_mapbox(
    communes,
    lat = "lat",
    lon = "lon",
    color = "classe",
    hover_data = ['libele'],
    size_max = 20,
    zoom = 8,
    color_continuous_scale = px.colors.cyclical.IceFire,
    mapbox_style = "open-street-map"
)
fig.show()